# 2. Preprocessing

## 2.1.Data Cleaning and Preparation

### 2.1.1. Data Aggregation

### Transaction Dataset

In [ ]:
#We started with the reduction of the transaction
import pandas as pd
import numpy as np
data_transaction=pd.read_csv("transactions.csv")
data_train=pd.read_csv("train_csv")
#First we are inner joining train dataset with our transaction dataset
merged_train=pd.merge(data_train, data_transaction, on='msno')
#Second in order to reduce the dataset even more we decided to take last six months transactions
reduced_merged_train=merged_train[merged_train['transaction_date']>20160831]
#saving the dataset just in case
pd.to_csv('transaction_reduced.csv')
#Third, we applied the aggregate functions to reduce the dataset in a way that each line represents one customer
df_transactions_sample= pd.read_csv('transaction_reduced.csv')
def transaction_statistics(df):

    # step1: sort 
    df_sorted = df.sort_values(['transaction_date'],ascending=True)  
    # remind: .sort_values will change dtypes into float

    # step2: construct new 'transaction' with new varailables
    newDF = {}
   
    newDF['avg_plan_list_price']=int(df_sorted['plan_list_price'].mean())
    newDF['avg_actual_amount_paid']=int(df_sorted['actual_amount_paid'].mean())
    newDF['total_cancel']=int(df_sorted['is_cancel'].sum())
    newDF['most_fq_payment_method_id']=int(df_sorted['payment_method_id'].value_counts().idxmax())
    newDF['most_frq_payment_plan_days']=int(df_sorted['payment_plan_days'].value_counts().idxmax())
  
    newDF['msno']=df_sorted.iloc[0]['msno']
    newDF['is_auto_renew']=int(df_sorted.iloc[-1]['is_auto_renew'])
    newDF['first_transaction_date']=int(df_sorted.iloc[0]['transaction_date'])
    newDF['last_expiration_date']=int(df_sorted.iloc[-1]['membership_expire_date'])
    newDF['total_churn']=total_churn(df)
    
     #compute number of active days for the customer
    first_activity=str(newDF['first_transaction_date'])
    last_activity=str(newDF['last_expiration_date'])
    first_activity_datetime=datetime.datetime.strptime(first_activity,'%Y%m%d')
    last_activity_datetime=datetime.datetime.strptime(last_activity,'%Y%m%d')
    active_days=last_activity_datetime-first_activity_datetime
    
    newDF['active_days']=active_days.days
    return newDF

# computing the number of times a customer has churned based on his transaction history
#input df = the relevant rows of one customer 
import datetime

def total_churn(df):

    df['index_col'] = range(0, len(df))
    df['index_col']
    i = 1 
    count = len(df)  
    total_churn = 0
    df['index_col'] = df.index

    while i < count:
        
        cur_tra_date = df.iloc[i]['transaction_date'].astype('str')
        cur_tra_datetime = datetime.datetime.strptime(cur_tra_date, '%Y%m%d')
        
        pre_exp_date = df.iloc[i-1]['membership_expire_date'].astype('str')
        pre_exp_datetime = datetime.datetime.strptime(pre_exp_date, '%Y%m%d')

        churn_days = cur_tra_datetime- pre_exp_datetime
        #print(churn_days.days)
    
    
        if churn_days.days > 30:
            total_churn += 1
    
        i += 1

    return total_churn


### remove noisy data
def de_noisy(df):
    # total remove 200k
    ds_nosiy = df.query('payment_plan_days<1 or plan_list_price<1 or actual_amount_paid<1')
    #ds_nosiy = df[(df['payment_plan_days']<1) | (df['plan_list_price']<1) | (df['actual_amount_paid']<1)]
    df_wo_noisy = df.drop(ds_nosiy.index)
    return df_wo_noisy


#de_noisy(df_transactions_sample[0:200]).head(140)


# ### final output in DataFrame
transaction_new=de_noisy(df_transactions_sample).groupby('msno').apply(transaction_statistics) # output three individual Dictionary! 
print("Group By is done")

# integrate all customer dictionaries into one super dic.
super_dic = {}
for c in transaction_new:
    for k, v in c.items():  # d.items() in Python 3+
        super_dic.setdefault(k, []).append(v)
        
# show as a Table (DataFrame)
super_transaction = pd.DataFrame(super_dic,columns=super_dic.keys())
super_transaction.reset_index(level=0, inplace=True)
super_transaction.to_csv("transactions_done.csv")

### UserLog Dataset

In [ ]:
#Now we started with the reduction of the userlogs
userlogs=pd.read_csv("userlogs.csv")
#This time first we reduced the dataset
reduced_userlogs=userlogs[userlogs['date']>20160831]
#Second inner joined it with train dataset
data_train=pd.read_csv("train_csv")
userlogs_reduced=pd.merge(data_train, reduced_userlogs, on='msno')
#Here we saved it just in case
userlogs_reduced.to_csv("userlog_reduced.csv")
#Third, we applied the aggregate functions to reduce the dataset in a way that each line represents one customer.
#Also we created specific variables that will represent specific months before the churn
csv=pd.read_csv("userlog_reduced.csv", dtype={'date':object})
csv.head()
csv.describe()
csv.dtypes

#subsettig year, month, day from the date variable and making the date numeric
i=0
nrows=csv.shape[0]
x = np.zeros((nrows,4))
while(i<nrows):
    s=csv['date'][i]
    x[i,0]=int(s[0:4])
    x[i,1]=int(s[4:6])
    x[i,2]=int(s[6:8])
    x[i,3]=int(s[0:8])
    i=i+1
csv=csv.assign(year=x[:,0],month=x[:,1],day=x[:,2],date_numeric=x[:,3])
print(csv.head())
print(csv.dtypes)
print("Month,day is DONE")

#creating sum, mean, median, max and min of variables
def f(x):
    d = {}
    d['num_25_sum'] = x['num_25'].sum()
    d['num_50_sum'] = x['num_50'].sum()
    d['num_75_sum'] = x['num_75'].sum()
    d['num_985_sum'] = x['num_985'].sum()
    d['num_100_sum'] = x['num_100'].sum()
    d['num_unq_sum'] = x['num_unq'].sum()
    d['num_totalsec_sum'] = x['total_secs'].sum()
    d['num_25_mean'] = x['num_25'].mean()
    d['num_50_mean'] = x['num_50'].mean()
    d['num_75_mean'] = x['num_75'].mean()
    d['num_985_mean'] = x['num_985'].mean()
    d['num_100_mean'] = x['num_100'].mean()
    d['num_unq_mean'] = x['num_unq'].mean()
    d['num_totalsec_mean'] = x['total_secs'].mean()
    d['num_25_med'] = x['num_25'].median()
    d['num_50_med'] = x['num_50'].median()
    d['num_75_med'] = x['num_75'].median()
    d['num_985_med'] = x['num_985'].median()
    d['num_100_med'] = x['num_100'].median()
    d['num_unq_med'] = x['num_unq'].median()
    d['num_totalsec_med'] = x['total_secs'].median()
    d['num_25_max'] = x['num_25'].max()
    d['num_50_max'] = x['num_50'].max()
    d['num_75_max'] = x['num_75'].max()
    d['num_985_max'] = x['num_985'].max()
    d['num_100_max'] = x['num_100'].max()
    d['num_unq_max'] = x['num_unq'].max()
    d['num_totalsec_max'] = x['total_secs'].max()
    d['num_25_min'] = x['num_25'].min()
    d['num_50_min'] = x['num_50'].min()
    d['num_75_min'] = x['num_75'].min()
    d['num_985_min'] = x['num_985'].min()
    d['num_100_min'] = x['num_100'].min()
    d['num_unq_min'] = x['num_unq'].min()
    d['num_totalsec_min'] = x['total_secs'].min()
    d['number_of_days_listened'] = x['date'].count()
    return pd.Series(d, index=["num_25_sum","num_50_sum","num_75_sum","num_985_sum","num_100_sum","num_unq_sum","num_totalsec_sum","num_25_mean","num_50_mean","num_75_mean","num_985_mean","num_100_mean","num_unq_mean","num_totalsec_mean","num_25_med","num_50_med","num_75_med","num_985_med","num_100_med","num_unq_med","num_totalsec_med","num_25_max","num_50_max","num_75_max","num_985_max","num_100_max","num_unq_max","num_totalsec_max","num_25_min","num_50_min","num_75_min","num_985_min","num_100_min","num_unq_min","num_totalsec_min","number_of_days_listened"])
data=csv.groupby('msno').apply(f)

# resetting the index

data.reset_index(level=0, inplace=True)

#Calculating churn variables

#how many times one person has listened in 201702 specific month. If not it should be zero
def f2017_02(x):
    d = {}
    d['num_25_201702_sum'] = x['num_25'].sum()
    d['num_50_201702_sum'] = x['num_50'].sum()
    d['num_75_201702_sum'] = x['num_75'].sum()
    d['num_985_201702_sum'] = x['num_985'].sum()
    d['num_100_201702_sum'] = x['num_100'].sum()
    d['num_unq_201702_sum'] = x['num_unq'].sum()
    d['num_totalsec_201702_sum'] = x['total_secs'].sum()
    d['num_25_201702_mean'] = x['num_25'].mean()
    d['num_50_201702_mean'] = x['num_50'].mean()
    d['num_75_201702_mean'] = x['num_75'].mean()
    d['num_985_201702_mean'] = x['num_985'].mean()
    d['num_100_201702_mean'] = x['num_100'].mean()
    d['num_unq_201702_mean'] = x['num_unq'].mean()
    d['num_totalsec_201702_mean'] = x['total_secs'].mean()
    d['number_of_days_201702_listened'] = x['date'].count()
    return pd.Series(d, index=["num_25_201702_sum","num_50_201702_sum","num_75_201702_sum","num_985_201702_sum","num_100_201702_sum","num_unq_201702_sum","num_totalsec_201702_sum","num_25_201702_mean","num_50_201702_mean","num_75_201702_mean","num_985_201702_mean","num_100_201702_mean","num_unq_201702_mean","num_totalsec_201702_mean","number_of_days_201702_listened"])
#how many times one person has listened in 201701 specific month. If not it should be zero
def f2017_01(x):
    d = {}
    d['num_25_201701_sum'] = x['num_25'].sum()
    d['num_50_201701_sum'] = x['num_50'].sum()
    d['num_75_201701_sum'] = x['num_75'].sum()
    d['num_985_201701_sum'] = x['num_985'].sum()
    d['num_100_201701_sum'] = x['num_100'].sum()
    d['num_unq_201701_sum'] = x['num_unq'].sum()
    d['num_totalsec_201701_sum'] = x['total_secs'].sum()
    d['num_25_201701_mean'] = x['num_25'].mean()
    d['num_50_201701_mean'] = x['num_50'].mean()
    d['num_75_201701_mean'] = x['num_75'].mean()
    d['num_985_201701_mean'] = x['num_985'].mean()
    d['num_100_201701_mean'] = x['num_100'].mean()
    d['num_unq_201701_mean'] = x['num_unq'].mean()
    d['num_totalsec_201701_mean'] = x['total_secs'].mean()
    d['number_of_days_201701_listened'] = x['date'].count()
    return pd.Series(d, index=["num_25_201701_sum","num_50_201701_sum","num_75_201701_sum","num_985_201701_sum","num_100_201701_sum","num_unq_201701_sum","num_totalsec_201701_sum","num_25_201701_mean","num_50_201701_mean","num_75_201701_mean","num_985_201701_mean","num_100_201701_mean","num_unq_201701_mean","num_totalsec_201701_mean","number_of_days_201701_listened"])
#how many times one person has listened in 201612 specific month. If not it should be zero
def f2016_12(x):
    d = {}
    d['num_25_201612_sum'] = x['num_25'].sum()
    d['num_50_201612_sum'] = x['num_50'].sum()
    d['num_75_201612_sum'] = x['num_75'].sum()
    d['num_985_201612_sum'] = x['num_985'].sum()
    d['num_100_201612_sum'] = x['num_100'].sum()
    d['num_unq_201612_sum'] = x['num_unq'].sum()
    d['num_totalsec_201612_sum'] = x['total_secs'].sum()
    d['num_25_201612_mean'] = x['num_25'].mean()
    d['num_50_201612_mean'] = x['num_50'].mean()
    d['num_75_201612_mean'] = x['num_75'].mean()
    d['num_985_201612_mean'] = x['num_985'].mean()
    d['num_100_201612_mean'] = x['num_100'].mean()
    d['num_unq_201612_mean'] = x['num_unq'].mean()
    d['num_totalsec_201612_mean'] = x['total_secs'].mean()
    d['number_of_days_201612_listened'] = x['date'].count()
    return pd.Series(d, index=["num_25_201612_sum","num_50_201612_sum","num_75_201612_sum","num_985_201612_sum","num_100_201612_sum","num_unq_201612_sum","num_totalsec_201612_sum","num_25_201612_mean","num_50_201612_mean","num_75_201612_mean","num_985_201612_mean","num_100_201612_mean","num_unq_201612_mean","num_totalsec_201612_mean","number_of_days_201612_listened"])
#applying the function
data201702=csv[(csv['year']==2017) & (csv['month'] == 2)].groupby('msno').apply(f2017_02)
data201701=csv[(csv['year']==2017) & (csv['month'] == 1)].groupby('msno').apply(f2017_01)
data201612=csv[(csv['year']==2016) & (csv['month'] == 12)].groupby('msno').apply(f2016_12)
#fixing the indexes
data201702.reset_index(level=0, inplace=True)
data201701.reset_index(level=0, inplace=True)
data201612.reset_index(level=0, inplace=True)

result=data.merge(data201702,on='msno',how='left').merge(data201701,on='msno',how='left').merge(data201612,on='msno',how='left')
print(result.head())

# Summing up the months of february and january 
def flasttwomonths(x):
    d = {}
    d['num_25_lasttwo_sum'] = x['num_25'].sum()
    d['num_50_lasttwo_sum'] = x['num_50'].sum()
    d['num_75_lasttwo_sum'] = x['num_75'].sum()
    d['num_985_lasttwo_sum'] = x['num_985'].sum()
    d['num_100_lasttwo_sum'] = x['num_100'].sum()
    d['num_unq_lasttwo_sum'] = x['num_unq'].sum()
    d['num_totalsec_lasttwo_sum'] = x['total_secs'].sum()
    d['num_25_lasttwo_mean'] = x['num_25'].mean()
    d['num_50_lasttwo_mean'] = x['num_50'].mean()
    d['num_75_lasttwo_mean'] = x['num_75'].mean()
    d['num_985_lasttwo_mean'] = x['num_985'].mean()
    d['num_100_lasttwo_mean'] = x['num_100'].mean()
    d['num_unq_lasttwo_mean'] = x['num_unq'].mean()
    d['num_totalsec_lasttwo_mean'] = x['total_secs'].mean()
    d['number_of_days_lasttwo_listened'] = x['date'].count()
    return pd.Series(d, index=["num_25_lasttwo_sum","num_50_lasttwo_sum","num_75_lasttwo_sum","num_985_lasttwo_sum","num_100_lasttwo_sum","num_unq_lasttwo_sum","num_totalsec_lasttwo_sum","num_25_lasttwo_mean","num_50_lasttwo_mean","num_75_lasttwo_mean","num_985_lasttwo_mean","num_100_lasttwo_mean","num_unq_lasttwo_mean","num_totalsec_lasttwo_mean","number_of_days_lasttwo_listened"])

#summing up the months of january february and december
def flastthreemonths(x):
    d = {}
    d['num_25_lastthree_sum'] = x['num_25'].sum()
    d['num_50_lastthree_sum'] = x['num_50'].sum()
    d['num_75_lastthree_sum'] = x['num_75'].sum()
    d['num_985_lastthree_sum'] = x['num_985'].sum()
    d['num_100_lastthree_sum'] = x['num_100'].sum()
    d['num_unq_lastthree_sum'] = x['num_unq'].sum()
    d['num_totalsec_lastthree_sum'] = x['total_secs'].sum()
    d['num_25_lastthree_mean'] = x['num_25'].mean()
    d['num_50_lastthree_mean'] = x['num_50'].mean()
    d['num_75_lastthree_mean'] = x['num_75'].mean()
    d['num_985_lastthree_mean'] = x['num_985'].mean()
    d['num_100_lastthree_mean'] = x['num_100'].mean()
    d['num_unq_lastthree_mean'] = x['num_unq'].mean()
    d['num_totalsec_lastthree_mean'] = x['total_secs'].mean()
    d['number_of_days_lastthree_listened'] = x['date'].count()
    return pd.Series(d, index=["num_25_lastthree_sum","num_50_lastthree_sum","num_75_lastthree_sum","num_985_lastthree_sum","num_100_lastthree_sum","num_unq_lastthree_sum","num_totalsec_lastthree_sum","num_25_lastthree_mean","num_50_lastthree_mean","num_75_lastthree_mean","num_985_lastthree_mean","num_100_lastthree_mean","num_unq_lastthree_mean","num_totalsec_lastthree_mean","number_of_days_lastthree_listened"])

#applying the function
#last two months calculation (feb+january)
datalasttwomonths=csv[(csv['date_numeric']<20170301) & (csv['date_numeric'] > 20161230)].groupby('msno').apply(flasttwomonths)
#last three months calculation (feb+january+december)
datalastthreemonths=csv[(csv['date_numeric']<20170301) & (csv['date_numeric'] > 20161130)].groupby('msno').apply(flastthreemonths)

#fixing the indexes
datalasttwomonths.reset_index(level=0, inplace=True)
datalastthreemonths.reset_index(level=0, inplace=True)

result=result.merge(datalasttwomonths,on='msno',how='left').merge(datalastthreemonths,on='msno',how='left')
print(result.head())
print(result.dtypes)

#fixing the empty columns. Because the if the customer hasn't listened any song within the last three months, the technique we use creates null columns 
result=result.fillna(0)
print("Non empty columns are fixed")
result.to_csv("user_logs_done.csv")

### 2.1.2. Merging the Datasets

In [ ]:
#Merging all the datasets
#We realized that not all the customers that are in the train dataset are in the member dataset. Therefore, first we decided to inner join members with 
#train. After joining transaction and member datasets, we are going to left join the merged dataset with transaction and userlogs datasets.
userlogs=pd.read_csv("user_logs_done.csv")
member=pd.read_csv("members.csv")
train=pd.read_csv("train.csv")
transaction=pd.read_csv("transactions_done.csv")
member_train=pd.merge(member,train, on='msno')
mem_tra_userog=pd.merge(member_train,userlogs, on='msno', how='left')
all=pd.merge(mem_tra_userog,transaction, on='msno', how='left')
#looking at the last dataset
all.shape
all.dtypes
all.describe()
#saving the data
all.to_csv("last_data_v1")

### 2.1.3. Data Cleaning

### Missing Handling for Userlog Variables

In [ ]:
#Missing Handling-> filling variables that came from userlog with 0
all=pd.read_csv("last_data_v1.csv")
all["num_25_sum"]=all["num_25_sum"].fillna(0)
all["num_50_sum"]=all["num_50_sum"].fillna(0)
all["num_75_sum"]=all["num_75_sum"].fillna(0)
all["num_985_sum"]=all["num_985_sum"].fillna(0)
all["num_100_sum"]=all["num_100_sum"].fillna(0)
all["num_unq_sum"]=all["num_unq_sum"].fillna(0)
all["num_totalsec_sum"]=all["num_totalsec_sum"].fillna(0)
all["num_25_mean"]=all["num_25_mean"].fillna(0)
all["num_50_mean"]=all["num_50_mean"].fillna(0)
all["num_75_mean"]=all["num_75_mean"].fillna(0)
all["num_985_mean"]=all["num_985_mean"].fillna(0)
all["num_100_mean"]=all["num_100_mean"].fillna(0)
all["num_unq_mean"]=all["num_unq_mean"].fillna(0)
all["num_totalsec_mean"]=all["num_totalsec_mean"].fillna(0)
all["num_25_med"]=all["num_25_med"].fillna(0)
all["num_50_med"]=all["num_50_med"].fillna(0)
all["num_75_med"]=all["num_75_med"].fillna(0)
all["num_985_med"]=all["num_985_med"].fillna(0)
all["num_100_med"]=all["num_100_med"].fillna(0)
all["num_unq_med"]=all["num_unq_med"].fillna(0)
all["num_totalsec_med"]=all["num_totalsec_med"].fillna(0)
all["num_25_max"]=all["num_25_max"].fillna(0)
all["num_50_max"]=all["num_50_max"].fillna(0)
all["num_75_max"]=all["num_75_max"].fillna(0)
all["num_985_max"]=all["num_985_max"].fillna(0)
all["num_100_max"]=all["num_100_max"].fillna(0)
all["num_unq_max"]=all["num_unq_max"].fillna(0)
all["num_totalsec_max"]=all["num_totalsec_max"].fillna(0)
all["num_25_min"]=all["num_25_min"].fillna(0)
all["num_50_min"]=all["num_50_min"].fillna(0)
all["num_75_min"]=all["num_75_min"].fillna(0)
all["num_985_min"]=all["num_985_min"].fillna(0)
all["num_100_min"]=all["num_100_min"].fillna(0)
all["num_unq_min"]=all["num_unq_min"].fillna(0)
all["num_totalsec_min"]=all["num_totalsec_min"].fillna(0)
all["number_of_days_listened"]=all["number_of_days_listened"].fillna(0)
all["num_25_201702_sum"]=all["num_25_201702_sum"].fillna(0)
all["num_50_201702_sum"]=all["num_50_201702_sum"].fillna(0)
all["num_75_201702_sum"]=all["num_75_201702_sum"].fillna(0)
all["num_985_201702_sum"]=all["num_985_201702_sum"].fillna(0)
all["num_100_201702_sum"]=all["num_100_201702_sum"].fillna(0)
all["num_unq_201702_sum"]=all["num_unq_201702_sum"].fillna(0)
all["num_totalsec_201702_sum"]=all["num_totalsec_201702_sum"].fillna(0)
all["num_25_201702_mean"]=all["num_25_201702_mean"].fillna(0)
all["num_50_201702_mean"]=all["num_50_201702_mean"].fillna(0)
all["num_75_201702_mean"]=all["num_75_201702_mean"].fillna(0)
all["num_985_201702_mean"]=all["num_985_201702_mean"].fillna(0)
all["num_100_201702_mean"]=all["num_100_201702_mean"].fillna(0)
all["num_unq_201702_mean"]=all["num_unq_201702_mean"].fillna(0)
all["num_totalsec_201702_mean"]=all["num_totalsec_201702_mean"].fillna(0)
all["number_of_days_201702_listened"]=all["number_of_days_201702_listened"].fillna(0)
all["num_25_201701_sum"]=all["num_25_201701_sum"].fillna(0)
all["num_50_201701_sum"]=all["num_50_201701_sum"].fillna(0)
all["num_75_201701_sum"]=all["num_75_201701_sum"].fillna(0)
all["num_985_201701_sum"]=all["num_985_201701_sum"].fillna(0)
all["num_100_201701_sum"]=all["num_100_201701_sum"].fillna(0)
all["num_unq_201701_sum"]=all["num_unq_201701_sum"].fillna(0)
all["num_totalsec_201701_sum"]=all["num_totalsec_201701_sum"].fillna(0)
all["num_25_201701_mean"]=all["num_25_201701_mean"].fillna(0)
all["num_50_201701_mean"]=all["num_50_201701_mean"].fillna(0)
all["num_75_201701_mean"]=all["num_75_201701_mean"].fillna(0)
all["num_985_201701_mean"]=all["num_985_201701_mean"].fillna(0)
all["num_100_201701_mean"]=all["num_100_201701_mean"].fillna(0)
all["num_unq_201701_mean"]=all["num_unq_201701_mean"].fillna(0)
all["num_totalsec_201701_mean"]=all["num_totalsec_201701_mean"].fillna(0)
all["number_of_days_201701_listened"]=all["number_of_days_201701_listened"].fillna(0)
all["num_25_201612_sum"]=all["num_25_201612_sum"].fillna(0)
all["num_50_201612_sum"]=all["num_50_201612_sum"].fillna(0)
all["num_75_201612_sum"]=all["num_75_201612_sum"].fillna(0)
all["num_985_201612_sum"]=all["num_985_201612_sum"].fillna(0)
all["num_100_201612_sum"]=all["num_100_201612_sum"].fillna(0)
all["num_unq_201612_sum"]=all["num_unq_201612_sum"].fillna(0)
all["num_totalsec_201612_sum"]=all["num_totalsec_201612_sum"].fillna(0)
all["num_25_201612_mean"]=all["num_25_201612_mean"].fillna(0)
all["num_50_201612_mean"]=all["num_50_201612_mean"].fillna(0)
all["num_75_201612_mean"]=all["num_75_201612_mean"].fillna(0)
all["num_985_201612_mean"]=all["num_985_201612_mean"].fillna(0)
all["num_100_201612_mean"]=all["num_100_201612_mean"].fillna(0)
all["num_unq_201612_mean"]=all["num_unq_201612_mean"].fillna(0)
all["num_totalsec_201612_mean"]=all["num_totalsec_201612_mean"].fillna(0)
all["number_of_days_201612_listened"]=all["number_of_days_201612_listened"].fillna(0)
all["num_25_lasttwo_sum"]=all["num_25_lasttwo_sum"].fillna(0)
all["num_50_lasttwo_sum"]=all["num_50_lasttwo_sum"].fillna(0)
all["num_75_lasttwo_sum"]=all["num_75_lasttwo_sum"].fillna(0)
all["num_985_lasttwo_sum"]=all["num_985_lasttwo_sum"].fillna(0)
all["num_100_lasttwo_sum"]=all["num_100_lasttwo_sum"].fillna(0)
all["num_unq_lasttwo_sum"]=all["num_unq_lasttwo_sum"].fillna(0)
all["num_totalsec_lasttwo_sum"]=all["num_totalsec_lasttwo_sum"].fillna(0)
all["num_25_lasttwo_mean"]=all["num_25_lasttwo_mean"].fillna(0)
all["num_50_lasttwo_mean"]=all["num_50_lasttwo_mean"].fillna(0)
all["num_75_lasttwo_mean"]=all["num_75_lasttwo_mean"].fillna(0)
all["num_985_lasttwo_mean"]=all["num_985_lasttwo_mean"].fillna(0)
all["num_100_lasttwo_mean"]=all["num_100_lasttwo_mean"].fillna(0)
all["num_unq_lasttwo_mean"]=all["num_unq_lasttwo_mean"].fillna(0)
all["num_totalsec_lasttwo_mean"]=all["num_totalsec_lasttwo_mean"].fillna(0)
all["number_of_days_lasttwo_listened"]=all["number_of_days_lasttwo_listened"].fillna(0)
all["num_25_lastthree_sum"]=all["num_25_lastthree_sum"].fillna(0)
all["num_50_lastthree_sum"]=all["num_50_lastthree_sum"].fillna(0)
all["num_75_lastthree_sum"]=all["num_75_lastthree_sum"].fillna(0)
all["num_985_lastthree_sum"]=all["num_985_lastthree_sum"].fillna(0)
all["num_100_lastthree_sum"]=all["num_100_lastthree_sum"].fillna(0)
all["num_unq_lastthree_sum"]=all["num_unq_lastthree_sum"].fillna(0)
all["num_totalsec_lastthree_sum"]=all["num_totalsec_lastthree_sum"].fillna(0)
all["num_25_lastthree_mean"]=all["num_25_lastthree_mean"].fillna(0)
all["num_50_lastthree_mean"]=all["num_50_lastthree_mean"].fillna(0)
all["num_75_lastthree_mean"]=all["num_75_lastthree_mean"].fillna(0)
all["num_985_lastthree_mean"]=all["num_985_lastthree_mean"].fillna(0)
all["num_100_lastthree_mean"]=all["num_100_lastthree_mean"].fillna(0)
all["num_unq_lastthree_mean"]=all["num_unq_lastthree_mean"].fillna(0)
all["num_totalsec_lastthree_mean"]=all["num_totalsec_lastthree_mean"].fillna(0)
all["number_of_days_lastthree_listened"]=all["number_of_days_lastthree_listened"].fillna(0)
#dropping one unrelated variable
all=all.drop(["Unnamed: 0_y","index"],axis=1)
#saving it again
all.to_csv("last_data_v3.csv")


### Missing Handling & Data Cleaning for Transaction Variables

In [ ]:
import pandas as pd
import numpy as np
#Missing Handling Continues
data=pd.read_csv("last_data_v3.csv")
print(data.shape)
print(data.columns)
#checking the dataset
print(data['num_totalsec_max'].min())
#data=data.drop['Unnamed: 0_y','index','first_transaction_date','last_expiration_date']
#handling missing values with gender
from sklearn.preprocessing import LabelEncoder
data['gender'] = LabelEncoder().fit_transform(data['gender'].astype(str)) #male==1, NAN==2,female==0
data['gender'] = LabelEncoder().fit_transform(data['gender'].astype(np.int8))

# find most 'frequent paymen_method_id' and 'most_frq_payment_plan_days' of non-null data
most_freq_paymethod=data['most_fq_payment_method_id'].value_counts().idxmax()
most_freq_payplandays=data['most_frq_payment_plan_days'].value_counts().idxmax()
most_freq_autorenew=data['is_auto_renew'].value_counts().idxmax()
most_freq_totcancel=data['total_cancel'].value_counts().idxmax()
most_freq_planlist=data['avg_plan_list_price'].value_counts().idxmax()
most_freq_actamount=data['avg_actual_amount_paid'].value_counts().idxmax()
print(most_freq_paymethod)
print(most_freq_payplandays)
print(most_freq_autorenew)
print(most_freq_totcancel)
print(most_freq_planlist)
print(most_freq_actamount)

In [ ]:
data['most_fq_payment_method_id']=data['most_fq_payment_method_id'].fillna(most_freq_paymethod) #41
data['most_frq_payment_plan_days']=data['most_frq_payment_plan_days'].fillna(most_freq_payplandays) #30
data['total_churn']=data['total_churn'].fillna(0)
data['is_auto_renew']=data['is_auto_renew'].fillna(most_freq_autorenew) #1
data['total_cancel']=data['total_cancel'].fillna(most_freq_totcancel) #0
data['active_days']=data['active_days'].fillna(0)
data['avg_plan_list_price']=data['avg_plan_list_price'].fillna(most_freq_planlist) #149
data['avg_actual_amount_paid']=data['avg_actual_amount_paid'].fillna(most_freq_actamount) #149

#create new variable
#calculate payment diference: 
data['payment_different']=data['avg_actual_amount_paid']-data['avg_plan_list_price']
data['num_below_50_sum'] = data['num_25_sum'] + data['num_50_sum'] 
data['num_above_50_sum'] = data['num_75_sum'] + data['num_985_sum']+data['num_100_sum']
data['total_songs_listened']=data['num_25_sum'] + data['num_50_sum']+ data['num_75_sum'] + data['num_985_sum']+data['num_100_sum']
data['proportion_songs_above_50']=data['num_above_50_sum']/data['total_songs_listened']

#Even though we handled with missing values, due to the fact that the denomater has 0 values, python couldn't do the division and created missing values
data['proportion_songs_above_50']=data['proportion_songs_above_50'].fillna(0)

#dtypes getting the whole list
data_dtypes=data.dtypes
data_dtypes.to_csv("dtypes_v3.csv")

#dropping unrelated variables
data=data.drop(["Unnamed: 0","Unnamed: 0.1","Unnamed: 0_x","registration_init_time","expiration_date","first_transaction_date","last_expiration_date"],axis=1)

#checking missingness-- No miissing values found.
i=0
n=data.shape[1]
column=data.columns
while(i<n):
    value=data[column[i]].isnull().values.any()
    if(value==True):
        print(column[i])
    i=i+1

In [ ]:
#Changing seconds to hours
data["num_totalsec_sum"]=data["num_totalsec_sum"]/3600
data["num_totalsec_mean"]=data["num_totalsec_mean"]/3600
data["num_totalsec_med"]=data["num_totalsec_med"]/3600
data["num_totalsec_max"]=data["num_totalsec_max"]/3600
data["num_totalsec_min"]=data["num_totalsec_min"]/3600
data["num_totalsec_201702_sum"]=data["num_totalsec_201702_sum"]/3600
data["num_totalsec_201702_mean"]=data["num_totalsec_201702_mean"]/3600
data["num_totalsec_201701_sum"]=data["num_totalsec_201701_sum"]/3600
data["num_totalsec_201701_mean"]=data["num_totalsec_201701_mean"]/3600
data["num_totalsec_201612_sum"]=data["num_totalsec_201612_sum"]/3600
data["num_totalsec_201612_mean"]=data["num_totalsec_201612_mean"]/3600
data["num_totalsec_lasttwo_sum"]=data["num_totalsec_lasttwo_sum"]/3600
data["num_totalsec_lasttwo_mean"]=data["num_totalsec_lasttwo_mean"]/3600
data["num_totalsec_lastthree_sum"]=data["num_totalsec_lastthree_sum"]/3600
data["num_totalsec_lastthree_mean"]=data["num_totalsec_lastthree_mean"]/3600


#checking maximum values
data_desc=data.describe()
data_desc.to_csv("data_desc_v3.csv")

#looking at the mean value of age
age_normal = data.query(' 0<bd<81')
print(age_normal.shape)
print(age_normal['bd'].mean())

In [ ]:
#handling outliers in age
data.ix[data.bd <0, 'bd'] = 0
data.ix[(data.bd >0) & (data.bd < 10) , 'bd'] = 29.74059164332968
data.ix[data.bd >80, 'bd'] = 29.74059164332968

#Active_days handling
data.loc[(data.active_days > -10) & (data.active_days < 0), 'active_days'] = 0
data=data.drop(data[data.active_days < -10].index)  # drop exreme ourlier value
data.to_csv("last_data_v5_prioroutliers")